# Use `seacarb`'s `derivnum` routine to compute sensitivities

<hr>
James Orr - 27 February 2018<br>

<img align="left" width="60%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png" \><br><br>

LSCE/IPSL, CEA-CNRS-UVSQ, Gif-sur-Yvette, France
<hr>

#### Table of Contents:
    1. Setup, install seacarb (one-time only), load seacarb (once per session)
    2. Compute sensitivities with derivnum routine


## 1. Setup 

#### Run interactively

If you are visualizing this after clicking on the link to this file on github, you are visualizing the HTML version of a jupyter notebook. Alternatively, you may run cells interactively and modify them if you have `jupyter notebook` installed on your machine.  To install that software, just download and the anaconda open software installer for your computing platform (Windows, OS X, or Linux) from https://www.continuum.io/downloads and then follow the easy install instructions at

https://docs.continuum.io/anaconda/install#

Then just download this `jupyter notebook` file as well as the seacarb package (see section 1.2 below).  Afterwards, you'll only need to install `Rmagic`, available in the `rpy2` package with the 2 lines of commands shown in the following section.

#### Install `R` and `Rmagic`

Before launching the notebook with the usual command `jupyter notebook`, it is necessary to install the Rmagic functionality.  That is now in the `rpy2` package of conda.  So to install that, at the Unix prompt, just enter the following 2 commands:

`conda install -c r r-essentials`

`conda install rpy2`

That only needs to be done once. Now go to the directory with this notebook, and at the Unix prompt, give the `jupyter notebook` command

#### Documentation for `Rmagic`

Details on using Rmagic are here: https://ipython.org/ipython-doc/2/config/extensions/rmagic.html

### 1.1 Allow use of the `Rmagic` extension (through the `rpy2` package)

In [1]:
%load_ext rpy2.ipython

### 1.2  Install seacarb and call library (need to install `seacarb` only once unlesss updating versions)

#### Uncomment last line in cell below and execute cell if you want to install or reinstall `seacarb` (no need to repeat this operation)

In [2]:
%%R
#install.packages('seacarb')
install.packages('seacarb', repos="http://ftp.igh.cnrs.fr/pub/CRAN/")

NULL


#### Use seacarb library (only need to invoke this once per session)

In [3]:
%%R
library(seacarb)

#### Version number of seacarb that you are using

In [4]:
%%R
packageVersion('seacarb')

[1] ‘3.2.8’


## 2.  Compute sensitivities (partial derivatives) with `derivnum` routine

In [5]:
%%R
?derivnum

R Help on ‘derivnum’derivnum                package:seacarb                R Documentation

_N_u_m_e_r_i_c_a_l _d_e_r_i_v_a_t_i_v_e_s _o_f _s_e_a_w_a_t_e_r _c_a_r_b_o_n_a_t_e _s_y_s_t_e_m _v_a_r_i_a_b_l_e_s

_D_e_s_c_r_i_p_t_i_o_n:

     Returns numerical derivatives of the seawater carbonate system
     output variables with respect to input variables.

_U_s_a_g_e:

     derivnum(varid, flag, var1, var2, S=35, T=25, Patm=1, P=0, Pt=0, Sit=0,
            k1k2="x", kf="x", ks="d", pHscale="T", b="u74", gas="potential", warn="y", 
            eos = "eos80", long = 1e+20, lat = 1e+20)
     
_A_r_g_u_m_e_n_t_s:

   varid: Variable length, case insensitive, character identificator of
          variable with respect to which derivatives are requested.
          Possible values are:

          '1' or 'var1' : Variable 1 of the input pair (This is TAlk if
          flag is 15)

          '2' or 'var2' : Variable 2 o

### 2.1 `carb` routine computes derived variables but does not compute sensitivities

In [6]:
%%R
# Standard seacarb routine to compute derived variables (carb, not errors)
vars <- carb(flag=15, var1=2300e-6, var2=2000e-6, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")

print(vars)

  flag  S  T Patm P       pH          CO2     fCO2     pCO2  fCO2pot  pCO2pot
1   15 35 18    1 0 8.152525 1.018509e-05 297.0486 298.0854 297.0854 298.1224
  fCO2insitu pCO2insitu        HCO3          CO3   DIC    ALK OmegaAragonite
1   297.0486   298.0854 0.001779217 0.0002105975 0.002 0.0023       3.252296
  OmegaCalcite
1     5.029397


### 2.2 Use the `derivnum` routine to compute uncertainties

The `derivnum` routine is used with the same arguments as the `carb` routine except that a new argument is inserted at the very beginning to specify the the input variable with respect to which the partial derivatives of all output variables will be taken.

#### 2.2.1 For the $A_\text{T}$-$C_\text{T}$ pair  (var1 and var2 when flag=15) compute derivatives of all output variables with respect to DIC (i.e., var2)


In [7]:
%%R
derivnum(varid='var2', flag=15, var1=2300e-6, var2=2000e-6, S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0,
         pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")
# shown are dH/dDIC, dpH/dDIC, dCO2*/dDIC, dfCO2/dDIC, dpCO2/dDIC, dHCO3/dDIC, dCO3/dDIC, dOmegaAragonite/dDIC,...

             H       pH        CO2    fCO2    pCO2     HCO3        CO3
1 3.511675e-05 -1695.41 0.05389665 1898312 1904392 1.588404 -0.6423006
  OmegaAragonite OmegaCalcite
1      -10193.63    -15465.18


#### 2.2.2 For the pH-$A_\text{T}$ pair, compute derivatives of all output vars w.r.t. [H+] (i.e., since var1 is pH)

In [8]:
%%R
d <- derivnum('1', flag=8, var1=8.1, var2=2300e-6, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            pHscale="T", kf="dg", k1k2="l", ks="d", b="u74")

d
# shown are dCO2/dH, dfCO2/dH, dpCO2/dH, dHCO3/dH, dCO3/dH, dDIC/dH, dOmegaAragonite/dH, dOmegaCalcite/dH

       CO2        fCO2        pCO2     HCO3       CO3     DIC OmegaAragonite
1 1791.739 52256164574 52438644528 47414.67 -19142.11 30064.3     -295615129
  OmegaCalcite
1   -457143454


#### 2.2.3 For the pH-$A_\text{T}$ pair, compute derivatives of all output vars w.r.t. the equil constant K1

In [9]:
%%R
d <- derivnum('K1', flag=8, var1=8.1, var2=2300e-6, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            pHscale="T", kf="dg", k1k2="l", ks="d", b="u74")

d
# shown are dCO2/dH, dfCO2/dH, dpCO2/dH, dHCO3/dH, dCO3/dH, dDIC/dH, dOmegaAragonite/dH, ...

        CO2       fCO2       pCO2          HCO3           CO3       DIC
1 -9.596446 -279880895 -280858247 -2.710505e-10 -2.258755e-11 -9.596446
  OmegaAragonite  OmegaCalcite
1  -1.850372e-07 -3.700743e-07


#### 2.2.4 For the pH-$A_\text{T}$ pair, compute derivatives of all output vars w.r.t. the equil constant Kspa

In [10]:
%%R
d <- derivnum('Kspa', flag=8, var1=8.1, var2=2300e-6, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            pHscale="T", kf="dg", k1k2="l", ks="d", b="u74")

d
# shown are dCO2/dKspa, dfCO2/dKspa, dpCO2/dKspa, dHCO3/dKspa, dCO3/dKspa, dDIC/dKspa, dOmegaAragonite/dKspa, ...

  CO2 fCO2 pCO2 HCO3 CO3 DIC OmegaAragonite OmegaCalcite
1   0    0    0    0   0   0       -4440977            0


#### 2.2.5 For the pH-$A_\text{T}$ pair, compute derivatives of all output vars w.r.t. Temperature

In [11]:
%%R
d <- derivnum('T', flag=8, var1=8.1, var2=2300e-6, S=35, T=18, P=0, Patm=1.0, Pt=0, Sit=0, 
            pHscale="T", kf="dg", k1k2="l", ks="d", b="u74")

d
# shown are dCO2/dT, dfCO2/dT, dpCO2/dT, dHCO3/dT, dCO3/dT, dDIC/dT, dOmegaAragonite/dT

            CO2      fCO2       pCO2         HCO3          CO3           DIC
1 -3.512524e-07 -0.365013 -0.3813695 -1.38782e-05 5.800958e-06 -8.428496e-06
  OmegaAragonite OmegaCalcite
1     0.09897379    0.1419354


#### 2.2.6 For other examples of using seacarb's `derivnum` routine

Just cut & paste what you like from the output of the cell below into a new jupyter notebook cell & try them out, interactively

In [12]:
%%R
example(derivnum)


dervnm> ## 1) For the input pair ALK and DIC (var1 and var2 when flag=15)
dervnm> ##    compute derivatives of all output varialbes 
dervnm> ##    with respect to DIC (i.e., var2)
dervnm> derivnum(varid='var2', flag=15, var1=2300e-6, var2=2000e-6, 
dervnm+          S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0,
dervnm+          pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")
             H       pH        CO2    fCO2    pCO2     HCO3        CO3
1 3.511675e-05 -1695.41 0.05389665 1898312 1904392 1.588404 -0.6423006
  OmegaAragonite OmegaCalcite
1      -10193.63    -15465.18

dervnm> ## 2) For the input pair pH and ALK (var1 and var2 when flag=8)
dervnm> ##    compute derivatives of all output variables 
dervnm> ##    with respect to [H+] concentration
dervnm> derivnum(varid='var1', flag=8, var1=8.2, var2=0.00234, 
dervnm+          S=35, T=25, P=0, Patm=1.0, Pt=0, Sit=0,
dervnm+          pHscale="T", kf="pf", k1k2="l", ks="d", b="u74")
       CO2       fCO2        pCO2     HCO3       CO3     